UNIVERSIDADE ESTADUAL DO OESTE DO PARANÁ - UNIOESTE
PROGRAMA DE PÓS GRADUAÇÃO EM CIÊNCIA DA COMPUTAÇÃO – PPGCOMP


Estruturas de Dados e Análise de Algoritmos - EDAA



Avaliação 1.1 – Algoritmos de Busca – Parte 1


##1) Descrição
Esta atividade individual consiste em implementar e comparar empiricamente a eficiência dos seguintes métodos de busca considerando arranjos estáticos com valores inteiros:
* Busca sequencial padrão;
* Busca por saltos (jump search);
* Busca binária.


## 2) Casos de Teste
Para realizar a comparação, devem ser gerados diferentes cenários de teste aleatórios variando-se o tamanho do arranjo de 100.000 a 1.000.000, em intervalos de 100 mil.

Calcular média e desvio padrão para o número de comparações e o tempo de execução considerando:
* O pior caso, com 3 execuções de cada;
* Casos aleatórios: 100 buscas para cada cenário.

## 3) Execução
* A linguagem de programação é livre;
* Para preencher o arranjo, pode ser usado um método pseudoaleatório random) disponível na linguagem mantendo-se a seed em cada cenário para os diferentes algoritmos comparados (uma função de shuffle também pode ser utilizada);
 * Os valores não devem se repetir;
 * O custo de criação do arranjo deve ser descartado;
* O custo de ordenação, quando necessário, deve ser computado no custo total, mas registrado
e discutido individualmente.
 *  Utilize um método de ordenação disponível na linguagem;
* Se o tempo de execução for muito pequeno, pode-se incluir um custo constante em cada
comparação da chave de busca com o elemento do arranjo.

## 4) Entrega
Os resultados devem ser apresentados em até 4 (quatro) páginas em PDF no formato de artigos da SBC – Sociedade Brasileira de Computação disponível em [Modelos para Publicação de Artigos](https://www.sbc.org.br/documentos-da-sbc/category/169-templates-para-artigos-e-capitulos-de-livros). Incluir Resumo/Abstract, Introdução, Materiais e Métodos, Resultados, Conclusão e Referências bibliográficas.

## Configuração do ambiente

In [ ]:
# Listar locales instalados
!locale -a

C
C.UTF-8
en_US.utf8
POSIX


In [ ]:
# Instalar o locale pt_BR
!/usr/share/locales/install-language-pack pt_BR
!dpkg-reconfigure locales

Generating locales (this might take a while)...
  pt_BR.ISO-8859-1... done
Generation complete.
dpkg-trigger: error: must be called from a maintainer script (or with a --by-package option)

Type dpkg-trigger --help for help about this utility.
Generating locales (this might take a while)...
  en_US.UTF-8... done
  pt_BR.ISO-8859-1... done
Generation complete.


In [ ]:
# Listar locales instalados
!locale -a

C
C.UTF-8
en_US.utf8
POSIX
pt_BR
pt_BR.iso88591


In [ ]:
# Reinicar o processo do python para enxergar o locale pt_BR
import os
os.kill(os.getpid(), 9)

In [ ]:
# Ignorar alerta de itens depreciados
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Definir o locale pt_BR
import locale
locale.setlocale(locale.LC_NUMERIC, 'pt_BR.ISO8859-1')

'pt_BR.ISO8859-1'

## Código

In [ ]:
# Definições
TAM_ARRANJO_MIN =   100_000
TAM_ARRANJO_MAX = 1_000_000
TAM_ARRANJO_INC =   100_000

EXEC_DESCATAR_MAIOR = 2
EXEC_ORDENACAO = 3 + EXEC_DESCATAR_MAIOR
EXEC_PIOR_CASO = 3 + EXEC_DESCATAR_MAIOR
EXEC_BUSCAS = 100 + EXEC_DESCATAR_MAIOR

FATOR_ESPALHAMENTO = 100

In [ ]:
# pacotes utilizados
import sys                                  # Aborta ao detectar um resultado inesperado
import random                               # Utilizado na geração do arranjo e da chave de busca - inteiros (pseudo)aleatórios
import pandas as pd                         # Utilizado para calcular os resultados agregados das execuções

from math import sqrt                       # Utilizado no cálculo do tamanho do salto/bloco - busca_por_saltos
from collections import namedtuple          # Utilizado para armazenar os resultados da execução de cada busca realizada
from collections.abc import Sequence        # Type hint nas funções que aceitam arranjos
from timeit import default_timer as timer   # Utilizado na temporização das buscas e ordenações
from types import FunctionType              # Type hint na função executar_busca, que aceita uma função metodo (de busca) como argumento
from typing import Union                    # Type hint nas funções de busca, que retornam um int ou None

In [ ]:
# variáveis globais
resultados = list()

# semente de aleatoriedade fixa para assegurar reprodutibilidade
random.seed(42)                             # o sentido da vida, o universo e tudo mais

In [ ]:
def gerar_arranjo(tamanho: int) -> Sequence:
  '''
  Gera um arranjo aleatório não ordenado com tamanho elementos e valores entre 0 e o tamanho do arranjo * o fator de espalhamento
  '''
  return random.sample(range(tamanho * FATOR_ESPALHAMENTO), tamanho)

In [ ]:
def gerar_chave_aleatoria(arranjo: Sequence) -> int:
  '''
  Gera uma chave de busca escolhendo, com igual probabilidade, entre:
  um valor qualquer entre 0 e o tamanho do arranjo * o fator de espalhamento e
  um valor qualquer presente no arranjo
  Probabilidade da chave estar presente no arranjo = 1/fator espalhamento + 0,5
  '''
  chave_existente = random.choice(arranjo)
  return random.choice([random.randrange(len(arranjo) * FATOR_ESPALHAMENTO), chave_existente])

In [ ]:
def gerar_chave_pior_caso(arranjo: Sequence) -> int:
  '''
  Dado um um arranjo, gera uma chave que corresponde ao pior caso para todos os algoritmos de busca considerados
  '''
  # Busca sequencial: Percorrer toda o arranjo (chave último elemento ou chave inexistente)
  # Busca por saltos: Percorrer todo o arranjo (chave último elemento)
  # Busca binária   : Percorrer todo o arranjo (chave último elemento ou maior que o último)
  chave = arranjo[-1]  # Chave de pesquisa igual ao último elemento
  return chave

In [ ]:
Resultado = namedtuple('Resultado', [
    'tamanho_arranjo',
    'rodada',
    'min',
    'max',
    'chave',
    'presente',
    'metodo',
    'comparacoes',
    'tempo',
    'posicao',
    'cenario'
])

In [ ]:
Resultado.__doc__ += ': Armazena o resultado de uma execução de busca em arranjo'
Resultado.tamanho_arranjo.__doc__ = 'Tamanho do arranjo no qual se efetuou a busca'
Resultado.rodada.__doc__ = 'Número da rodada de testes'
Resultado.min.__doc__ = 'Menor valor no arranjo'
Resultado.max.__doc__ = 'Maior valor no arranjo'
Resultado.chave.__doc__ = 'Chave buscada'
Resultado.presente.__doc__ = 'Indica se o valor da chave está presente ou não no arranjo'
Resultado.metodo.__doc__ = 'Método de busca utilizado'
Resultado.comparacoes.__doc__ = 'Número de comparações efetuados'
Resultado.tempo.__doc__ = 'Tempo gasto na busca/ordenação'
Resultado.posicao.__doc__ = 'Posição na qual a chave de busca foi encontrada'
Resultado.cenario.__doc__ = 'Cenário da execução ordenação ou busca: aleatório ou pior caso'

In [ ]:
def registrar_resultado(resultado: Resultado):
  '''
  Salva um resultado individual na lista de resultados
  '''
  global resultados
  resultados.append(resultado)

In [ ]:
def busca_sequencial(arranjo: Sequence, chave: int, **kwargs) -> tuple[Union[int, None], int]:
  '''
  Implementação do algoritmo da busca sequencial em arranjos.
  Percorre todo o arranjo, comparando cada elemento elemento com a chave de busca.
  Retorna a posição da chave no arranjo ou None caso o elemento não esteja presente no arranjo.
  Parâmetros obrigatórios:
  arranjo: sequência de elementos
  chave: chave de busca
  Parâmetros opcionais (nomeadods):
  esquerda: posição inicial da busca (default, 0)
  direita: posição final da busca (default, tamanho do arranjo)
  '''
  comparacoes = 0
  esquerda = kwargs.get('esquerda', 0)
  direita  = kwargs.get('direita' , len(arranjo))

  for indice, elemento in zip(range(esquerda, direita), arranjo[esquerda:direita]):
    comparacoes += 1
    if elemento == chave:
      return (indice, comparacoes)

  return (None, comparacoes) # Não encontrou

In [ ]:
def busca_por_saltos(arranjo: Sequence, chave: int, **kwargs) -> tuple[Union[int, None], int]:
  '''
  Implementação do algoritmo da busca sequencial em arranjos.
  Tamanho do salto/bloco fixo: raiz quadrada do tamanho do arranjo.
  Percorre o arranjo, saltando para frente de salto em salto, até encontrar um elemento maior ou igual à chave de busca.
  Quando isso acontece, salta para trás salto elementos e realiza uma busca sequência no bloco.
  Retorna a posição da chave no arranjo ou None caso o elemento não esteja presente no arranjo.
  Parâmetros obrigatórios:
  arranjo: sequência de elementos ordenados
  chave: chave de busca
  Parâmetros opcionais (nomeadods):
  ordenado: flag indicando se o arranjo está ordenado ou não (default, não ordenado)
  esquerda: posição inicial da busca (default, 0)
  direita: posição final da busca (default, tamanho do arranjo)
  '''
  comparacoes = 0
  esquerda = kwargs.get('esquerda', 0)
  direita  = kwargs.get('direita' , len(arranjo))

  ordenado = kwargs.get('ordenado', False)
  if not ordenado:
    raise ValueError('arranjo deve estar ordenado')

  salto = int(sqrt(direita - esquerda))

  ultimo_elemento_bloco = min(esquerda + salto, direita - 1)
  while ultimo_elemento_bloco < direita:
    comparacoes += 1
    if arranjo[ultimo_elemento_bloco] >= chave:
      inicio_bloco = max(0, ultimo_elemento_bloco - salto + 1)
      posicao, comparacoes_seq = busca_sequencial(arranjo, chave, esquerda=inicio_bloco, direita=ultimo_elemento_bloco + 1)
      return (posicao, comparacoes + comparacoes_seq)
    ultimo_elemento_bloco = min(ultimo_elemento_bloco + salto, direita - 1)

  return (None, comparacoes) # Não encontrou

In [ ]:
def busca_binaria(arranjo: Sequence, chave: int, **kwargs) -> tuple[Union[int, None], int]:
  '''
  Implementação  do algoritmo de busca binária em arranjos.
  Divide sucessivamente o arranjo ao meio, comparando elemento central com a chave, caso este seja:
  igual à chave    : retorna a posição
  maior que a chave: busca no segmento inferior [esquerda, meio - 1]
  menor que a chave: busca no segmento superior [meio + 1, direita ]
  Parâmetros obrigatórios:
  arranjo: sequência de elementos ordenados
  chave: chave de busca
  Parâmetros opcionais (nomeadods):
  ordenado: flag indicando se o arranjo está ordenado ou não (default, não ordenado)
  esquerda: posição inicial da busca (default, 0)
  direita: posição final da busca (default, tamanho do arranjo - 1)
  '''
  comparacoes = 0
  esquerda = kwargs.get('esquerda', 0)
  direita  = kwargs.get('direita' , len(arranjo) - 1)

  ordenado = kwargs.get('ordenado', False)
  if not ordenado:
    raise ValueError('arranjo deve estar ordenado')

  while esquerda <= direita:
    meio = esquerda + (direita - esquerda) // 2
    elemento = arranjo[meio]

    comparacoes += 1
    if chave == elemento:
      return (meio, comparacoes)
    elif chave < elemento:  # deve-se buscar na porção inferior
      direita = meio - 1
    else:
      esquerda = meio + 1   # deve-se buscar na porção superior

  return (None, comparacoes) # Não encontrou

In [ ]:
def executar_busca(arranjo: Sequence, chave: int, metodo: FunctionType, cenario: str, ordenado = False) -> tuple[Union[int, None], int]:
  '''
  Para um dado cenário, busca a chave no arranjo utilizando o método de busca fornecido
  e registra o resultado da execução.
  '''
  if ordenado:
    menor_valor = arranjo[0]
    maior_valor = arranjo[-1]
  else:
    menor_valor = min(arranjo)
    maior_valor = max(arranjo)

  tempo = timer()
  posicao, comparacoes = metodo(arranjo, chave, ordenado=ordenado)
  tempo = timer() - tempo

  registrar_resultado(Resultado(
      tamanho_arranjo=tamanho_arranjo,
      rodada=rodada,
      min=menor_valor,
      max=maior_valor,
      chave=chave,
      presente=chave in arranjo,
      metodo=metodo.__name__.replace('busca_', '').replace('_', ' ').replace('binaria', 'binária').capitalize(),
      comparacoes=comparacoes,
      tempo=tempo,
      posicao=posicao,
      cenario=cenario
  ))

  return (posicao, comparacoes)

In [ ]:
# Métodos de busca que serão avaliados
metodos_de_busca = (busca_sequencial, busca_por_saltos, busca_binaria)

In [ ]:
# Executa as buscas para os diversos tamanhos de arranjo
for tamanho_arranjo in range(TAM_ARRANJO_MIN, TAM_ARRANJO_MAX + 1, TAM_ARRANJO_INC):
  arranjo = gerar_arranjo(tamanho_arranjo)

  for rodada in range(0, EXEC_ORDENACAO):
    tempo_ordenacao = timer()
    arranjo_ordenado = sorted(arranjo)
    tempo_ordenacao = timer() - tempo_ordenacao
    registrar_resultado(Resultado(
        tamanho_arranjo=tamanho_arranjo,
        rodada=0,
        min=arranjo_ordenado[0],
        max=arranjo_ordenado[-1],
        chave=None,
        presente=None,
        metodo='TimSort',
        comparacoes=None,
        tempo=tempo_ordenacao,
        posicao=None,
        cenario='Ordenação'
    ))
  ordenado = True

  # Cenário: Pior Caso
  chave = gerar_chave_pior_caso(arranjo_ordenado)
  for rodada in range(0, EXEC_PIOR_CASO):
    for metodo in metodos_de_busca:
      posicao, _ = executar_busca(arranjo_ordenado, chave, metodo, cenario='Pior Caso', ordenado=ordenado)
      if posicao != tamanho_arranjo - 1:
        sys.exit(f'Resultado diferente do esperado - método {metodo.__name__}. Esperado {tamanho_arranjo - 1} - Encontrado: {posicao}')

  # Cenário: Aleatório
  for rodada in range(0, EXEC_BUSCAS):
    chave = gerar_chave_aleatoria(arranjo_ordenado)
    for metodo in metodos_de_busca:
      executar_busca(arranjo_ordenado, chave, metodo, cenario='Aleatório', ordenado=ordenado)

### Calcular as estatíticas

In [ ]:
df_bruto = pd.DataFrame(resultados) # Antes de descartar os maiores valores
df = df_bruto.copy()                # Será filtrado para descartar os maiores valores

In [ ]:
# Descartar os EXEC_DESCATAR_MAIOR maiores tempos - devido ao warm up e ao fato do ambiente ser compartilhado
get_tamanho_metodo_cenario = lambda df: df[['tamanho_arranjo', 'metodo', 'cenario']].drop_duplicates().itertuples(index=False)
get_max_tempo_idx = lambda df, tamanho, metodo, cenario: df[(df['cenario'] == cenario) & (df['metodo'] == metodo) & (df['tamanho_arranjo'] == tamanho)]['tempo'].idxmax()

for tamanho, metodo, cenario in get_tamanho_metodo_cenario(df_bruto):
  for _ in range(0, EXEC_DESCATAR_MAIOR):
    try:
      max_tempo_idx = get_max_tempo_idx(df, tamanho, metodo, cenario)
    except ValueError: # Caso ocorra é seguro ignorar
      pass
    else:
      df.drop(max_tempo_idx, inplace=True)

In [ ]:
# Separa os dados de ordenação dos dados de busa
df_ordenacao = df[df['cenario'] == 'Ordenação'].copy()
df_busca = df[df['cenario'] != 'Ordenação'].copy()

In [ ]:
# Transforma os valores booleanos (True e False) em inteiros (1 e 0, respectivamente)
# Para calcular o % médio de buscas nos quais a chave estava presente no arranjo
# Pois a chave ausente corresponde ao pior caso da busca sequencial, e dependendo do valor, da binária também
df_busca['presente'] = df_busca['presente'].replace([True, False], [int(True), int(False)])

In [ ]:
# Calcula as estatísticas descritivas: média, desvio padrão, mínimo, mediana (50% percentil) e máximo
# para as colunas tempo (de busca), número de comparações e % de buscas com chave pertencente ao arranjo
# dos resultados agrupados por tamanho do arranjo e método de busca (sequencial, por saltos e binária)
# serparada para os cenário de busca aleatório ou pior caso
df_aleatorio = df_busca[df_busca['cenario'] == 'Aleatório'].groupby(
    ['tamanho_arranjo', 'metodo']
)[['tempo', 'comparacoes', 'presente']].describe(percentiles=[])

# não selecionamos a coluna presente no cenário pior caso pois sempre será 1, não agregando informação
df_pior_caso = df_busca[df_busca['cenario'] == 'Pior Caso'].groupby(
    ['tamanho_arranjo', 'metodo']
)[['tempo', 'comparacoes']].describe(percentiles=[])

# só temos as medições de tempo
df_timsort = df_ordenacao[df_ordenacao['metodo'] == 'TimSort'].groupby(
    ['tamanho_arranjo', 'metodo']
)[['tempo']].describe(percentiles=[])

### Exibir resultados

In [ ]:
# Formatadores
# transforma o float de s para ms e aplica a formação de acordo com a localização
to_ms = lambda s: locale.format_string('%.2f', 1000*s, grouping=True)
# aplica a formação de acordo com a localização - inteiros e floats
locale_format_int = lambda d: locale.format_string('%d', d, grouping=True)
locale_format_float = lambda f: locale.format_string('%.2f', f, grouping=True)

In [ ]:
def renomear_coluna(coluna: str) -> str:
  if coluna == None:
    return None
  return coluna.\
    replace('tempo','Tempo (ms)').\
    replace('comparacoes','Comp.').\
    replace('presente', 'x ∈ A').\
    replace('mean','x̄').\
    replace('std', 'σ')

In [ ]:
# Prepara o DataFrame para exibição/apresentação
def reformatar_df(df: pd.DataFrame, cenario: Union[str,None] = None) -> pd.DataFrame:
  # remover colunas indesejadas
  colunas_selecionadas = [coluna for coluna in df.columns if coluna[1] not in ('count','min','50%', 'max')]
  novo_df = df[colunas_selecionadas].copy()

  # renomear colunas
  if cenario is None:
    colunas_renomeadas = [(renomear_coluna(medida), renomear_coluna(estatistica)) for medida,estatistica in novo_df.columns.values]
  else:   # adiciona o cenário ao nome das colunas - MultiIndex
    colunas_renomeadas = [(cenario, renomear_coluna(medida), renomear_coluna(estatistica)) for medida,estatistica in novo_df.columns.values]
  novo_df.columns = novo_df.columns.values
  novo_df.columns = pd.MultiIndex.from_tuples(colunas_renomeadas)

  # reformatar valores dos índices
  df_idx_names = novo_df.index.names
  df_idx_values = [(locale_format_int(tamanho), metodo) for tamanho,metodo in novo_df.index.values]
  novo_df.index = pd.MultiIndex.from_tuples(df_idx_values, names=df_idx_names)

  # renomear colunas dos índices
  novo_df.rename_axis(
    index={
      'tamanho_arranjo': 'Tam.',
      'metodo': 'Busca' if cenario is not None else 'Ord.'
    },
    inplace=True
  )

  return novo_df

In [ ]:
from pandas.io.formats.style import Styler
def exibir_df(df: pd.DataFrame) -> Styler:
  # associa os formatadores às colunas
  formatters={
    col:(to_ms if 'Tempo' in col[len(col) - 2] else
        locale_format_float if len(col)!=3 or col[0] != 'Pior Caso' else
        locale_format_int)
    for col in df.columns
  }

  # Exibe o df aplicando os formatadores
  return df.style.format(formatters)

In [ ]:
# remove colunas não informativas
df_estatisticas_busca = reformatar_df(df_aleatorio, 'Aleatório').join(reformatar_df(df_pior_caso, 'Pior Caso')).drop([
    ('Pior Caso', 'Comp.', 'σ'),  # Será sempre 0
    ('Aleatório', 'x ∈ A', 'σ'),        # Estamos interessados apenas na média, que nos informa o % de chaves pertencente ao arranjo
], axis=1)

In [ ]:
exibir_df(df_estatisticas_busca)

In [ ]:
# Visualizar resultados - ordenacao
df_estatisticas_ordenacao = reformatar_df(df_timsort)

exibir_df(df_estatisticas_ordenacao)

In [ ]:
df_estatisticas_unificado = df_estatisticas_busca.join(reformatar_df(df_timsort.reindex(index=df_aleatorio.index, method='backfill'), 'Ordenação'))

In [ ]:
exibir_df(df_estatisticas_unificado)

### Salvar resultados

In [ ]:
import os # Manipulação de FS

def to_latex(df: pd.DataFrame, hide_index: bool = False, **kwargs) -> str:
  if hide_index:
    new_str = exibir_df(df).hide_index().to_latex(hrules=True, **kwargs)
  else:
    new_str = exibir_df(df).to_latex(hrules=True, **kwargs)
  replacements = {
      'x̄': r'$\bar{x}$',
      'σ': r'$\sigma$',
      'x ∈ A': r'$x \in A$',
  }
  for old, new in replacements.items():
    new_str = new_str.replace(old, new)
  return new_str

def save_text(text: str, path: str):
  with open(path, mode='w') as fout:
    fout.write(text)

if not os.path.exists('./resultados'):
  os.mkdir('./resultados')

# Todas as execuções
df.to_csv('./resultados/resultados-todos.csv', sep=';')
df.style.to_excel('./resultados/resultados-todos.xlsx')
# Buscas
df_busca.to_csv('./resultados/resultados-busca.csv', sep=';')
df_busca.style.to_excel('./resultados/resultados-busca.xlsx')
# Ordenação
df_ordenacao.to_csv('./resultados/resultados-ordenacao.csv', sep=';')
df_ordenacao.style.to_excel('./resultados/resultados-ordenacao.xlsx')

# Gerar tabela latex - semi-formatada para inclusão no artigo
save_text(
  to_latex(
    df_estatisticas_unificado,
    clines='skip-last;data',
    caption='Comparação empírica do desempenho dos métodos de busca',
    label='tab:resultados'
  ),
  os.path.join('./resultados', 'tabela-unificada.tex')
)

In [ ]:
import shutil
from google.colab import files

shutil.make_archive('resultados', 'zip', 'resultados')

files.download('resultados.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Informações sobre o ambiente

In [ ]:
import platform

# Plataforma
print(platform.platform())
# Versão Python
print(platform.python_implementation() + ': ' + platform.python_version())
# Pandas
print(pd.show_versions())

Linux-5.10.147+-x86_64-with-glibc2.31
CPython: 3.9.16

INSTALLED VERSIONS
------------------
commit           : 2e218d10984e9919f0296931d92ea851c6a6faf5
python           : 3.9.16.final.0
python-bits      : 64
OS               : Linux
OS-release       : 5.10.147+
Version          : #1 SMP Sat Dec 10 16:00:40 UTC 2022
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : None
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.5.3
numpy            : 1.22.4
pytz             : 2022.7.1
dateutil         : 2.8.2
setuptools       : 67.7.2
pip              : 23.0.1
Cython           : 0.29.34
pytest           : 7.2.2
hypothesis       : None
sphinx           : 3.5.4
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.9.2
html5lib         : 1.1
pymysql          : None
psycopg2         : 2.9.6
jinja2           : 3.1.2
IPython          : 7.34.0
pandas_datareader: 0.10.0
bs4           

In [ ]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          12985        1437        8117          21        3430       11249
Swap:             0           0           0


In [ ]:
!cat /proc/meminfo

MemTotal:       13297192 kB
MemFree:         8310920 kB
MemAvailable:   11518668 kB
Buffers:          329732 kB
Cached:          3060240 kB
SwapCached:            0 kB
Active:           606360 kB
Inactive:        4144616 kB
Active(anon):       3480 kB
Inactive(anon):  1360072 kB
Active(file):     602880 kB
Inactive(file):  2784544 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              2160 kB
Writeback:             0 kB
AnonPages:       1360960 kB
Mapped:           315148 kB
Shmem:             21824 kB
KReclaimable:     123548 kB
Slab:             155120 kB
SReclaimable:     123548 kB
SUnreclaim:        31572 kB
KernelStack:        4096 kB
PageTables:        25652 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648596 kB
Committed_AS:    2828164 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        8964 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2200.214
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4400.42
clflush size	: 64
cache_alignment	: 64
addres

In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2200.154
BogoMIPS:                        4400.30
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 